In [7]:
import os
import re
from pymongo import MongoClient
import pandas as pd

In [8]:
MONGO_URI = os.getenv("MONGO_URI")
MONGO_DATABASE = os.getenv("MONGO_DATABASE")
pd.set_option('display.max_colwidth', 1000)

In [19]:
client = MongoClient(MONGO_URI)
db = client["DJs_v4"]
selected_keys = {"acordaoId": 1, "index": 1, "observacao": 1, "citacoesObs": 1, "publicacao": 1, "citacoes_revistas": 1}
page_ranks_cursor = db["acordaos"].find({}, selected_keys)
df = pd.DataFrame(page_ranks_cursor)

# Motivo de criação deste notebook

Este notebook foi usado para desenvolver a função que extrai citações e checar a acurácia dela.  
Para decisões dos anos [2001, 2019] a extração de citações de revistas não funcionou apenas para
duas citações que stão incorretas.  
Portanto, a função funciona muito bem.

In [20]:
df["publicacao"]

0                           DJ 03-05-2002 PP-00020  EMENT VOL-02067-04 PP-00715
1                           DJ 07-02-2003 PP-00044  EMENT VOL-02097-03 PP-00611
2                           DJ 19-12-2002 PP-00097  EMENT VOL-02096-04 PP-00840
3                           DJ 07-02-2003 PP-00024  EMENT VOL-02097-05 PP-00926
4                           DJ 19-12-2002 PP-00071  EMENT VOL-02096-12 PP-02656
                                          ...                                  
104013    PROCESSO ELETRÔNICO \r\nDJe-282  DIVULG 17-12-2019  PUBLIC 18-12-2019
104014    PROCESSO ELETRÔNICO \r\nDJe-282  DIVULG 17-12-2019  PUBLIC 18-12-2019
104015    PROCESSO ELETRÔNICO \r\nDJe-282  DIVULG 17-12-2019  PUBLIC 18-12-2019
104016    PROCESSO ELETRÔNICO \r\nDJe-282  DIVULG 17-12-2019  PUBLIC 18-12-2019
104017    PROCESSO ELETRÔNICO \r\nDJe-282  DIVULG 17-12-2019  PUBLIC 18-12-2019
Name: publicacao, Length: 104018, dtype: object

In [11]:
magazines_list = ["MAG-CD", "MAG-DVD", "RGJ-CD", "JTARS", "BIBJURID-DVD",
                  "JCSTF", "JPSTF", "JPO-DVD", "JBCC", "JBC", "JBT",
                  "JC", "JTJ", "JTJ-CD", "JURISonline-INT",
                  "JurisSíntese-DVD", "JurisSíntese-INT", "LEXJTACSP",
                  "LEXSTF", "LEXSTF-CD", "MAG-INT", "NRDF",
                  "PLENUMonline-INT", "REPIOB", "RADCOAST", "RB", "RCJ",
                  "RDA", "RDC", "RDP", "RDTAPET", "RDJTJDFT", "RDJ", "RET",
                  "RJDTACSP", "RJADCOAS", "RJTJRS", "RJTJRS-INT",
                  "RDECTRAB", "RDDP", "RDDT", "RMP", "RT", "RF", "RIP",
                  "RIOBTP", "RJTS", "RJSP", "RJMG", "RJP", "RJP-CD",
                  "REVJMG", "REVJMG-INT", "RLTR", "RMDPPP", "RNDJ",
                  "RPTGJ", "RSJADV", "RST", "RSTP", "RTFP", "RTJ",
                  "RTJE", "SINTESE-INT", "COAD-INT"
                 ]
print(len(magazines_list))
print(len(set(magazines_list)))
max_mag_size = max([len(mag) for mag in magazines_list])

65
65


In [14]:
for mag in magazines_list:
    publ_size = sum(df["publicacao"].apply(lambda x: 0 if re.search("\s+" + mag + "[^\w]+", x) is None else 1))
    dec_cit_size = sum(df["observacao"].apply(lambda x: 0 if re.search("\s+" + mag + "[^\w]+", x) is None else 1))
    if (publ_size > 0) or (dec_cit_size > 0):
        print(mag + " " * (max_mag_size - len(mag)), publ_size, " " * (4 - len(str(publ_size))), dec_cit_size)

JBC              3     0
JC               36    0
JTJ              0     11
LEXSTF           4095  19
REPIOB           5     0
RADCOAST         3     0
RB               107   0
RCJ              80    0
RDA              7     117
RDC              0     432
RDP              0     3
RDTAPET          2     0
RDJ              0     1
RET              27    0
RJDTACSP         1     1
RJADCOAS         34    0
RJTJRS           38    0
RDECTRAB         124   0
RDDP             98    0
RDDT             190   1
RMP              88    0
RT               1225  677
RF               95    183
RIP              34    0
RIOBTP           12    0
RJSP             69    4
RJP              52    0
REVJMG           73    0
RLTR             5     0
RMDPPP           41    1
RNDJ             119   0
RPTGJ            4     0
RSJADV           75    0
RSTP             17    0
RTFP             43    0
RTJ              2662  10439


In [13]:
print(df[df["observacao"].apply(lambda x: False if re.search("[^\w]+" + "JTJ" + "[^\w]+", x) is None else True)]["observacao"].values[0])
print(df[df["observacao"].apply(lambda x: False if re.search("[^\w]+" + "RDDT" + "[^\w]+", x) is None else True)]["observacao"].values[0])


- Acórdãos citados: Inq 777 QO (RTJ 153/760), Inq 1566, Inq 1879 (RTJ 190/834), HC 39495 (RTJ 24/313), HC 41590; RT 720/526; RDJ 4/227; JTJ 236/340; STJ: RHC 4593, HC 7376; TRF - 1ª Região: HC 19990100071817-8. - Veja HC 757 do Tribunal Regional Federal - 5ª Região. - Foi substituído o Ministro Relator de acordo com o art. 38, IV, "b", do RISTF. Número de páginas: 57 Análise: 22/04/2008, ACL. Revisão: 24/04/2008, ACL.
- Acórdãos citados: Pet 3593 QO (RT 96/493, LEXSTF 29/510, RDDT 140/205), HC 81611, HC 83414, RHC 83717, HC 84105, HC 85185, HC 86120 (LEXSTF 27/496), HC 88657 AgR, HC 89902 (RT 97/521, RDDT 147/159), HC 90957. Número de páginas: 10 Análise: 14/05/2008, IMC. Revisão: 14/05/2008, JBM.


In [21]:
df["magazines_count"] = df['publicacao'].apply(lambda x: sum([0 if re.search("\s+" + mag + "[^\w]+", x) is None
                                                             else 1 for mag in magazines_list]))

In [27]:
print(df["magazines_count"].value_counts())

df[df["magazines_count"] > 4]["publicacao"].values[0]

0    96617
1     5742
2     1322
3      276
4       55
5        6
Name: magazines_count, dtype: int64


'DJ 04-08-2006  PP-00078   EMENT VOL-02240-04 PP-00658\r\nRTJ VOL-00201-02 PP-00730\r\nRB v. 18, n. 515, 2006, p. 36-38\r\nLEXSTF v. 28, n. 333, 2006, p. 250-260\r\nRDDT n. 133, 2006, p. 176-180\r\nRET v. 9, n. 51, 2006, p. 90-97'

In [17]:
df[df["magazines_count"] > 4]["acordaoId"].values[0]

'RE 170784'

In [23]:
df["magazines_real_count"] = df["citacoes_revistas"].apply(lambda x: len(x))

In [24]:
df["magazines_real_count"].value_counts()

0    96619
1     5740
2     1322
3      276
4       55
5        6
Name: magazines_real_count, dtype: int64

In [30]:
df[df["magazines_real_count"] != df["magazines_count"]][["acordaoId", "publicacao", "citacoes_revistas"]]

,acordaoId,publicacao,citacoes_revistas
1258,ADI 1233,DJ 10-08-2001 PP-00002 EMENT VOL-02038-01 PP-00126\r\nRTJ VOL-00178/03 PP-01078,[]
1721,QO RE 113682,DJ 11-10-2001 PP-00018 EMENT VOL-02047-02 PP-00418\r\nRTJ-00182 T-01 PP-00298,[]


In [39]:
publication = "DJ 11-10-2001 PP-00018 EMENT VOL-02047-02 PP-00418\r\nRTJ VOL-00182-01 PP-00298"
re.search(
    "\s+" + "RTJ" + "[^\w]+.*VOL\-?\s*0*(\d+)\-?\d*\s*PP\-?0*(\d+)",
    publication,
).groups()
# citações deveriam ser: TJ VOL-00178-03 PP-01078
#                        RTJ VOL-00182-01 PP-00298

('182', '298')

In [46]:
# CITAÇÃO DE REVISTA CASA COM O PADRÃO?
q = "SÚMULA 35"
# acordaoId_pattern = "([^\d]{2,}[\s-]+\d+[^\d]*)$"
# re.search(acordaoId_pattern, q)
re.search("[\w]+", q)

<_sre.SRE_Match object; span=(0, 6), match='SÚMULA'>